In [ ]:
import os,sys, pathlib, time, subprocess
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import re
from pathlib import Path
from scipy.interpolate import UnivariateSpline



folder_path = pathlib.Path().resolve()
planing_path = folder_path.parent.parent
problem_path = folder_path / "Problemas"
print("Current folder path:", folder_path)
print("Planing folder path:", planing_path)
print("Problem folder path:", problem_path)

sizes = []
times = []
steps = []
costs = []
step_pattern = re.compile(r"\s*\d+\s*:")
cost_pattern = re.compile(r"plan cost:\s*([0-9]+(?:\.[0-9]+)?)", re.IGNORECASE)





# MetricFF

In [ ]:

max_time = 60  # Tiempo máximo en segundos para cada resolución
size = 2
time_exceeded = False
max_size = 0


if not problem_path.exists():
    os.makedirs(problem_path)

pbar = tqdm(desc="Resolviendo tamaños", unit="problema", position=0, leave=True)
while not time_exceeded:
    pbar.set_description(f"Tamaño actual: {size}")
    # Generar el problema con los parámetros actuales
    gen_cmd = f"python3 {folder_path}/generate_problem.py -d 1 -r 1 -l {size} -p {size} -c {size} -g {size} -v 0"
    # pbar.write(f"\n[INFO] Generando problema para tamaño {size}...")
    os.system(gen_cmd)
    pddl_files = sorted(problem_path.glob("*.pddl"), key=os.path.getmtime, reverse=True)
    if not pddl_files:
        pbar.write("[ERROR] No se encontró el archivo .pddl generado.")
        break
    problem_pddl = pddl_files[0]
    planificador_cmd = [
        f"{planing_path}/planificadores/metricff",
        "-o", f"{folder_path}/dron-domain.pddl",
        "-f", str(problem_pddl)
    ]
    
    start = time.time()
    try:
        result = subprocess.run(planificador_cmd, timeout=max_time, capture_output=True, text=True)
        elapsed = time.time() - start
        if result.returncode == 0:
            max_size = size
            sizes.append(size)
            times.append(elapsed)
            result_lines = result.stdout.splitlines()
            step_count = 1
            for line in result.stdout.splitlines():
                if step_pattern.match(line):
                    step_count += 1
            cost_match = cost_pattern.search(result.stdout)
            if cost_match:
                cost = float(cost_match.group(1))
                costs.append(int(cost))
            else:
                cost = None
                costs.append(None)
            steps.append(step_count)
            
            pbar.write(f"[OK] Plan encontrado para tamaño {size} en {elapsed:.4f} segundos con {step_count} pasos.")
            pbar.set_postfix({"Tiempo": f"{elapsed:.2f}s", "Tamaño": size})
            size += 1
            pbar.update(1)
        else:
            time_exceeded = True
    except subprocess.TimeoutExpired:
        pbar.write(f"[TIMEOUT] Tiempo excedido ({max_time}s) para tamaño {size}.")
        time_exceeded = True
        
    os.remove(problem_pddl)

pbar.close()
print(f"\n>>> El mayor tamaño resuelto en menos de 1 minuto es: {max_size}")



In [ ]:
print(sizes)
print(times)
print(steps)
print(costs)

In [ ]:
results_csv_path = folder_path / "Resultados/resultadosMetricff.csv"
if not results_csv_path.parent.exists():
    os.makedirs(results_csv_path.parent)

# Añadir la columna de costes
new_df = pd.DataFrame({'Tamaño': sizes, 'Tiempo (s)': times, 'Pasos': steps, 'Coste': costs})

if results_csv_path.exists():
    old_df = pd.read_csv(results_csv_path)
    # Si el archivo antiguo no tiene la columna 'Coste', la añadimos con NaN
    if 'Coste' not in old_df.columns:
        old_df['Coste'] = float('nan')
    combined_df = pd.concat([old_df, new_df], ignore_index=True)

    # Calcular la media de tiempo y coste por tamaño
    tiempo_df = combined_df.groupby('Tamaño', as_index=False)['Tiempo (s)'].mean()
    pasos_df = combined_df.groupby('Tamaño', as_index=False)['Pasos'].max()
    coste_df = combined_df.groupby('Tamaño', as_index=False)['Coste'].mean()

    # Unir todos los resultados en un solo DataFrame
    final_df = tiempo_df.merge(pasos_df, on='Tamaño').merge(coste_df, on='Tamaño')
else:
    final_df = new_df

final_df.to_csv(results_csv_path, index=False)
print(f"Resultados exportados a {results_csv_path}")


In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 5))

# Gráfico 1: Tiempo vs Tamaño
axs[0].scatter(sizes, times, marker='o', color='tab:blue')
axs[0].set_xlabel('Tamaño del problema')
axs[0].set_ylabel('Tiempo (s)')
axs[0].set_title('Tiempo vs Tamaño')
axs[0].grid(True)

# Gráfico 2: Pasos vs Tamaño
axs[1].scatter(sizes, steps, marker='s', color='tab:orange')
axs[1].set_xlabel('Tamaño del problema')
axs[1].set_ylabel('Número de pasos')
axs[1].set_title('Pasos vs Tamaño')
axs[1].grid(True)

# Gráfico 3: Coste vs Tamaño
axs[2].scatter(sizes, costs, marker='^', color='tab:green')
axs[2].set_xlabel('Tamaño del problema')
axs[2].set_ylabel('Coste')
axs[2].set_title('Coste vs Tamaño')
axs[2].grid(True)

plt.tight_layout()
plt.show()


# Lama First

In [ ]:
planner_name = "Fastdownard-Lama"
sas_output_path = folder_path / "sas_plan"
# Remove --sas-file when using --alias
planner_cmd = [
    "/home/javikugan/Documents/Universidad/Tercero/PlaniExt/Plani/planificadores/downward.sif",
    "--alias", "lama-first",
    str(folder_path / "dron-domain.pddl")
    # problem_pddl will be appended later
]

max_time = 60
max_size = 0
sizes = []
times = []
steps_list = []
costs = []
outputs = []

size = 2
time_exceeded = False

if not problem_path.exists():
    os.makedirs(problem_path)

pbar = tqdm(desc="Resolviendo tamaños", unit="problema", position=0, leave=True)
while not time_exceeded:
    pbar.set_description(f"Tamaño actual: {size}")
    # Generar el problema con los parámetros actuales
    gen_cmd = f"python3 {folder_path}/generate_problem.py -d 1 -r 1 -l {size} -p {size} -c {size} -g {size} -v 0"
    os.system(gen_cmd)
    pddl_files = sorted(problem_path.glob("*.pddl"), key=os.path.getmtime, reverse=True)
    if not pddl_files:
        break
    problem_pddl = pddl_files[0]
    
    # Construir el comando completo para el planificador
    full_cmd = planner_cmd + [str(problem_pddl)]
    
    pddl_files = sorted(
        problem_path.glob("*.pddl"),
        key=os.path.getmtime,
        reverse=True
    )
    
    if not pddl_files:
        pbar.write("[ERROR] No se encontró el archivo .pddl generado.")
        break
    problem_pddl = pddl_files[0]

    pbar.set_postfix({"Planificador": planner_name, "Tamaño": size})

    start = time.time()
    try:
        result = subprocess.run(full_cmd, timeout=max_time, capture_output=True, text=True)
        elapsed = time.time() - start
        if result.returncode == 0:
            plan_output = sas_output_path.read_text().strip()
            # Contar líneas válidas de plan (ignorando comentarios)
            num_steps = sum(
                1
                for line in plan_output.strip().split("\n")
                if line.strip() and not line.startswith(";") and "(" in line
            )

            # Leer el coste del plan si existe
            plan_cost = None
            for line in plan_output.splitlines():
                line = line.strip()
                if line.startswith(";") and "cost" in line:
                    # Formato esperado: ; cost = 12808 (general cost)
                    try:
                        plan_cost = float(line.split("=")[1].split()[0])
                    except Exception:
                        plan_cost = None
                    break

            sizes.append(size)
            times.append(elapsed)
            steps_list.append(num_steps - 1)
            costs.append(plan_cost)
            outputs.append(plan_output[:500])  # guardar solo primeros 500 caracteres
            max_size = size
            
        else:
            pbar.write(f"[ERROR] Planificador {planner_name} falló para tamaño {size}.")
            print(f"Salida del planificador:\n{result.stdout}")
            time_exceeded = True
            break
    except Exception as e:
        pbar.write(f"[ERROR] Excepción al ejecutar el planificador: {e}")
        time_exceeded = True
        break
    pbar.update(1)
    size += 1
    os.remove(problem_pddl)
pbar.close()

print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list,
    "Coste": costs
})

df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Coste": costs,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])


In [ ]:
print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list
})
if csv_path.exists():
    old_df = pd.read_csv(csv_path)
    combined_df = pd.concat([old_df, df], ignore_index=True)
    final_df = combined_df.groupby('Tamaño', as_index=False)['Tiempo (s)'].mean()
    if 'Pasos' in combined_df.columns:
        pasos_df = combined_df.groupby('Tamaño', as_index=False)['Pasos'].max()
        final_df = final_df.merge(pasos_df, on='Tamaño')
else:
    final_df = df
final_df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])

In [ ]:
print(sizes)
print(times)
print(steps_list)
print(costs)

In [ ]:
# Cargar los datos desde el archivo CSV exportado
results = pd.read_csv(folder_path / "Resultados/resultados_fastdownard-lama.csv")

sizes = results['Tamaño']
times = results['Tiempo (s)']
steps = results['Pasos']

fig, axs = plt.subplots(1, 3, figsize=(14, 5))

# Gráfico 1: Tiempo vs Tamaño (sin líneas entre puntos)
axs[0].scatter(sizes, times, marker='o', color='tab:blue')


axs[0].set_xlabel('Tamaño del problema')
axs[0].set_ylabel('Tiempo (s)')
axs[0].set_title('Tiempo vs Tamaño')
axs[0].grid(True)

# Gráfico 2: Pasos vs Tamaño (sin líneas entre puntos)
axs[1].scatter(sizes, steps, marker='s', color='tab:orange')

axs[1].set_xlabel('Tamaño del problema')
axs[1].set_ylabel('Número de pasos')
axs[1].set_title('Pasos vs Tamaño')
axs[1].grid(True)

axs[2].scatter(sizes, costs, marker='^', color='tab:green')
axs[2].set_xlabel('Tamaño del problema')
axs[2].set_ylabel('Coste')
axs[2].set_title('Coste vs Tamaño')
axs[2].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
planner_name = "Fastdownard-Autotune 2"
sas_output_path = folder_path / "sas_plan"
# Remove --sas-file when using --alias
planner_cmd = [
    "/home/javikugan/Documents/Universidad/Tercero/PlaniExt/Plani/planificadores/downward.sif",
    "--alias", "seq-sat-fd-autotune-2", "--plan-file", str(sas_output_path),
    str(folder_path / "dron-domain.pddl")
    # problem_pddl will be appended later
]


max_time = 60
max_size = 0
sizes = []
times = []
steps_list = []
costs = []
outputs = []

size = 2
time_exceeded = False

if not problem_path.exists():
    os.makedirs(problem_path)

pbar = tqdm(desc="Resolviendo tamaños", unit="problema", position=0, leave=True)
while not time_exceeded:
    pbar.set_description(f"Tamaño actual: {size}")
    # Generar el problema con los parámetros actuales
    gen_cmd = f"python3 {folder_path}/generate_problem.py -d 1 -r 1 -l {size} -p {size} -c {size} -g {size} -v 0"
    os.system(gen_cmd)
    pddl_files = sorted(problem_path.glob("*.pddl"), key=os.path.getmtime, reverse=True)
    if not pddl_files:
        break
    problem_pddl = pddl_files[0]
    
    # Construir el comando completo para el planificador
    full_cmd = planner_cmd + [str(problem_pddl)]
    
    pddl_files = sorted(
        problem_path.glob("*.pddl"),
        key=os.path.getmtime,
        reverse=True
    )
    
    if not pddl_files:
        pbar.write("[ERROR] No se encontró el archivo .pddl generado.")
        break
    problem_pddl = pddl_files[0]

    pbar.set_postfix({"Planificador": planner_name, "Tamaño": size})

    start = time.time()
    try:
        result = subprocess.run(full_cmd, timeout=max_time, capture_output=True, text=True)
        elapsed = time.time() - start
        if result.returncode == 0:
    # Encontrar el último archivo sas_plan generado
            sas_files = sorted(
                folder_path.glob("sas_plan*"),
                key=lambda f: int(f.name.split(".")[-1]) if f.name != "sas_plan" else 0,
                reverse=True
            )
            
            if not sas_files:
                pbar.write("[ERROR] No se encontró archivo de plan")
                continue
            
            latest_sas = sas_files[0]
            
            # Leer y procesar el último plan
            plan_output = latest_sas.read_text().strip()
            
            # Contar pasos válidos
            num_steps = sum(
                1 for line in plan_output.split("\n")
                if line.strip() and not line.startswith(";") and "(" in line
            )
            
            # Extraer coste del plan
            plan_cost = None
            for line in plan_output.splitlines():
                if line.startswith(";") and "cost" in line:
                    try:
                        plan_cost = float(line.split("=")[1].split()[0].strip())
                    except (IndexError, ValueError):
                        pass
                    break
                    
            # Eliminar el archivo después de procesarlo
            sizes.append(size)
            times.append(elapsed)
            steps_list.append(num_steps - 1)
            costs.append(plan_cost)
            outputs.append(plan_output)  # guardar solo primeros 500 caracteres
            max_size = size
            
        else:
            pbar.write(f"[ERROR] Planificador {planner_name} falló para tamaño {size}.")
            print(f"Salida del planificador:\n{result.stdout}")
            time_exceeded = True
            break
    except Exception as e:
        pbar.write(f"[ERROR] Excepción al ejecutar el planificador: {e}")
        time_exceeded = True
        break
    pbar.update(1)
    size += 1
    os.remove(problem_pddl)
pbar.close()

print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list,
    "Coste": costs
})

df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Coste": costs,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])



In [ ]:
print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list
})
if csv_path.exists():
    old_df = pd.read_csv(csv_path)
    combined_df = pd.concat([old_df, df], ignore_index=True)
    final_df = combined_df.groupby('Tamaño', as_index=False)['Tiempo (s)'].mean()
    if 'Pasos' in combined_df.columns:
        pasos_df = combined_df.groupby('Tamaño', as_index=False)['Pasos'].max()
        final_df = final_df.merge(pasos_df, on='Tamaño')
else:
    final_df = df
final_df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])


# Cargar los datos desde el archivo CSV exportado
results = pd.read_csv(csv_path)

sizes = results['Tamaño']
times = results['Tiempo (s)']
steps = results['Pasos']

# Manejar la columna de coste si existe
if 'Coste' in results.columns:
    costs = results['Coste']
else:
    import numpy as np
    costs = np.full(len(sizes), np.nan)

fig, axs = plt.subplots(1, 3, figsize=(14, 5))

# Gráfico 1: Tiempo vs Tamaño (sin líneas entre puntos)
axs[0].scatter(sizes, times, marker='o', color='tab:blue')


axs[0].set_xlabel('Tamaño del problema')
axs[0].set_ylabel('Tiempo (s)')
axs[0].set_title('Tiempo vs Tamaño')
axs[0].grid(True)

# Gráfico 2: Pasos vs Tamaño (sin líneas entre puntos)
axs[1].scatter(sizes, steps, marker='s', color='tab:orange')

axs[1].set_xlabel('Tamaño del problema')
axs[1].set_ylabel('Número de pasos')
axs[1].set_title('Pasos vs Tamaño')
axs[1].grid(True)

axs[2].scatter(sizes, costs, marker='^', color='tab:green')
axs[2].set_xlabel('Tamaño del problema')
axs[2].set_ylabel('Coste')
axs[2].set_title('Coste vs Tamaño')
axs[2].grid(True)

plt.tight_layout()
plt.show()



In [ ]:
planner_name = "Fastdownard-seq-sat-fdss-2"
sas_output_path = folder_path / "sas_plan"
# Remove --sas-file when using --alias
planner_cmd = [
    "/home/javikugan/Documents/Universidad/Tercero/PlaniExt/Plani/planificadores/downward.sif",
    "--alias", "seq-sat-fdss-2", "--plan-file", str(sas_output_path),
    str(folder_path / "dron-domain.pddl")
    # problem_pddl will be appended later
]


max_time = 60
max_size = 0
sizes = []
times = []
steps_list = []
costs = []
outputs = []

size = 2
time_exceeded = False

if not problem_path.exists():
    os.makedirs(problem_path)

pbar = tqdm(desc="Resolviendo tamaños", unit="problema", position=0, leave=True)
while not time_exceeded:
    pbar.set_description(f"Tamaño actual: {size}")
    # Generar el problema con los parámetros actuales
    gen_cmd = f"python3 {folder_path}/generate_problem.py -d 1 -r 1 -l {size} -p {size} -c {size} -g {size} -v 0"
    os.system(gen_cmd)
    pddl_files = sorted(problem_path.glob("*.pddl"), key=os.path.getmtime, reverse=True)
    if not pddl_files:
        break
    problem_pddl = pddl_files[0]
    
    # Construir el comando completo para el planificador
    full_cmd = planner_cmd + [str(problem_pddl)]
    
    pddl_files = sorted(
        problem_path.glob("*.pddl"),
        key=os.path.getmtime,
        reverse=True
    )
    
    if not pddl_files:
        pbar.write("[ERROR] No se encontró el archivo .pddl generado.")
        break
    problem_pddl = pddl_files[0]

    pbar.set_postfix({"Planificador": planner_name, "Tamaño": size})

    start = time.time()
    try:
        result = subprocess.run(full_cmd, timeout=max_time, capture_output=True, text=True)
        elapsed = time.time() - start
        if result.returncode == 0:
    # Encontrar el último archivo sas_plan generado
            sas_files = sorted(
                folder_path.glob("sas_plan*"),
                key=lambda f: int(f.name.split(".")[-1]) if f.name != "sas_plan" else 0,
                reverse=True
            )
            
            if not sas_files:
                pbar.write("[ERROR] No se encontró archivo de plan")
                continue
            
            latest_sas = sas_files[0]
            
            # Leer y procesar el último plan
            plan_output = latest_sas.read_text().strip()
            
            # Contar pasos válidos
            num_steps = sum(
                1 for line in plan_output.split("\n")
                if line.strip() and not line.startswith(";") and "(" in line
            )
            
            # Extraer coste del plan
            plan_cost = None
            for line in plan_output.splitlines():
                if line.startswith(";") and "cost" in line:
                    try:
                        plan_cost = float(line.split("=")[1].split()[0].strip())
                    except (IndexError, ValueError):
                        pass
                    break
                    
            # Eliminar el archivo después de procesarlo
            sizes.append(size)
            times.append(elapsed)
            steps_list.append(num_steps - 1)
            costs.append(plan_cost)
            outputs.append(plan_output)  # guardar solo primeros 500 caracteres
            max_size = size
            
        else:
            pbar.write(f"[ERROR] Planificador {planner_name} falló para tamaño {size}.")
            print(f"Salida del planificador:\n{result.stdout}")
            time_exceeded = True
            break
    except Exception as e:
        pbar.write(f"[ERROR] Excepción al ejecutar el planificador: {e}")
        time_exceeded = True
        break
    pbar.update(1)
    size += 1
    os.remove(problem_pddl)
pbar.close()

print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list,
    "Coste": costs
})

df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Coste": costs,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])



In [ ]:
print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list
})
if csv_path.exists():
    old_df = pd.read_csv(csv_path)
    combined_df = pd.concat([old_df, df], ignore_index=True)
    final_df = combined_df.groupby('Tamaño', as_index=False)['Tiempo (s)'].mean()
    if 'Pasos' in combined_df.columns:
        pasos_df = combined_df.groupby('Tamaño', as_index=False)['Pasos'].max()
        final_df = final_df.merge(pasos_df, on='Tamaño')
else:
    final_df = df
final_df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])


# Cargar los datos desde el archivo CSV exportado
results = pd.read_csv(csv_path)

sizes = results['Tamaño']
times = results['Tiempo (s)']
steps = results['Pasos']

# Manejar la columna de coste si existe
if 'Coste' in results.columns:
    costs = results['Coste']
else:
    import numpy as np
    costs = np.full(len(sizes), np.nan)

fig, axs = plt.subplots(1, 3, figsize=(14, 5))

# Gráfico 1: Tiempo vs Tamaño (sin líneas entre puntos)
axs[0].scatter(sizes, times, marker='o', color='tab:blue')


axs[0].set_xlabel('Tamaño del problema')
axs[0].set_ylabel('Tiempo (s)')
axs[0].set_title('Tiempo vs Tamaño')
axs[0].grid(True)

# Gráfico 2: Pasos vs Tamaño (sin líneas entre puntos)
axs[1].scatter(sizes, steps, marker='s', color='tab:orange')

axs[1].set_xlabel('Tamaño del problema')
axs[1].set_ylabel('Número de pasos')
axs[1].set_title('Pasos vs Tamaño')
axs[1].grid(True)

axs[2].scatter(sizes, costs, marker='^', color='tab:green')
axs[2].set_xlabel('Tamaño del problema')
axs[2].set_ylabel('Coste')
axs[2].set_title('Coste vs Tamaño')
axs[2].grid(True)

plt.tight_layout()
plt.show()



In [ ]:
planner_name = "Fastdownard-seq-opt-lmcut"
sas_output_path = folder_path / "sas_plan"
# Remove --sas-file when using --alias
planner_cmd = [
    "/home/javikugan/Documents/Universidad/Tercero/PlaniExt/Plani/planificadores/downward.sif",
    "--alias", "seq-opt-lmcut", "--plan-file", str(sas_output_path),
    str(folder_path / "dron-domain.pddl")
    # problem_pddl will be appended later
]


max_time = 60
max_size = 0
sizes = []
times = []
steps_list = []
costs = []
outputs = []

size = 2
time_exceeded = False

if not problem_path.exists():
    os.makedirs(problem_path)

pbar = tqdm(desc="Resolviendo tamaños", unit="problema", position=0, leave=True)
while not time_exceeded:
    pbar.set_description(f"Tamaño actual: {size}")
    # Generar el problema con los parámetros actuales
    gen_cmd = f"python3 {folder_path}/generate_problem.py -d 1 -r 1 -l {size} -p {size} -c {size} -g {size} -v 0"
    os.system(gen_cmd)
    pddl_files = sorted(problem_path.glob("*.pddl"), key=os.path.getmtime, reverse=True)
    if not pddl_files:
        break
    problem_pddl = pddl_files[0]
    
    # Construir el comando completo para el planificador
    full_cmd = planner_cmd + [str(problem_pddl)]
    
    pddl_files = sorted(
        problem_path.glob("*.pddl"),
        key=os.path.getmtime,
        reverse=True
    )
    
    if not pddl_files:
        pbar.write("[ERROR] No se encontró el archivo .pddl generado.")
        break
    problem_pddl = pddl_files[0]

    pbar.set_postfix({"Planificador": planner_name, "Tamaño": size})

    start = time.time()
    try:
        result = subprocess.run(full_cmd, timeout=max_time, capture_output=True, text=True)
        elapsed = time.time() - start
        if result.returncode == 0:
    # Encontrar el último archivo sas_plan generado
            sas_files = sorted(
                folder_path.glob("sas_plan*"),
                key=lambda f: int(f.name.split(".")[-1]) if f.name != "sas_plan" else 0,
                reverse=True
            )
            
            if not sas_files:
                pbar.write("[ERROR] No se encontró archivo de plan")
                continue
            
            latest_sas = sas_files[0]
            
            # Leer y procesar el último plan
            plan_output = latest_sas.read_text().strip()
            
            # Contar pasos válidos
            num_steps = sum(
                1 for line in plan_output.split("\n")
                if line.strip() and not line.startswith(";") and "(" in line
            )
            
            # Extraer coste del plan
            plan_cost = None
            for line in plan_output.splitlines():
                if line.startswith(";") and "cost" in line:
                    try:
                        plan_cost = float(line.split("=")[1].split()[0].strip())
                    except (IndexError, ValueError):
                        pass
                    break
                    
            # Eliminar el archivo después de procesarlo
            sizes.append(size)
            times.append(elapsed)
            steps_list.append(num_steps - 1)
            costs.append(plan_cost)
            outputs.append(plan_output)  # guardar solo primeros 500 caracteres
            max_size = size
            
        else:
            pbar.write(f"[ERROR] Planificador {planner_name} falló para tamaño {size}.")
            print(f"Salida del planificador:\n{result.stdout}")
            time_exceeded = True
            break
    except Exception as e:
        pbar.write(f"[ERROR] Excepción al ejecutar el planificador: {e}")
        time_exceeded = True
        break
    pbar.update(1)
    size += 1
    os.remove(problem_pddl)
pbar.close()

print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list,
    "Coste": costs
})

df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Coste": costs,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])



In [ ]:
print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list
})
if csv_path.exists():
    old_df = pd.read_csv(csv_path)
    combined_df = pd.concat([old_df, df], ignore_index=True)
    final_df = combined_df.groupby('Tamaño', as_index=False)['Tiempo (s)'].mean()
    if 'Pasos' in combined_df.columns:
        pasos_df = combined_df.groupby('Tamaño', as_index=False)['Pasos'].max()
        final_df = final_df.merge(pasos_df, on='Tamaño')
else:
    final_df = df
final_df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])


# Cargar los datos desde el archivo CSV exportado
results = pd.read_csv(csv_path)

sizes = results['Tamaño']
times = results['Tiempo (s)']
steps = results['Pasos']

# Manejar la columna de coste si existe
if 'Coste' in results.columns:
    costs = results['Coste']
else:
    import numpy as np
    costs = np.full(len(sizes), np.nan)

fig, axs = plt.subplots(1, 3, figsize=(14, 5))

# Gráfico 1: Tiempo vs Tamaño (sin líneas entre puntos)
axs[0].scatter(sizes, times, marker='o', color='tab:blue')


axs[0].set_xlabel('Tamaño del problema')
axs[0].set_ylabel('Tiempo (s)')
axs[0].set_title('Tiempo vs Tamaño')
axs[0].grid(True)

# Gráfico 2: Pasos vs Tamaño (sin líneas entre puntos)
axs[1].scatter(sizes, steps, marker='s', color='tab:orange')

axs[1].set_xlabel('Tamaño del problema')
axs[1].set_ylabel('Número de pasos')
axs[1].set_title('Pasos vs Tamaño')
axs[1].grid(True)

axs[2].scatter(sizes, costs, marker='^', color='tab:green')
axs[2].set_xlabel('Tamaño del problema')
axs[2].set_ylabel('Coste')
axs[2].set_title('Coste vs Tamaño')
axs[2].grid(True)

plt.tight_layout()
plt.show()



In [ ]:
print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list
})
if csv_path.exists():
    old_df = pd.read_csv(csv_path)
    combined_df = pd.concat([old_df, df], ignore_index=True)
    final_df = combined_df.groupby('Tamaño', as_index=False)['Tiempo (s)'].mean()
    if 'Pasos' in combined_df.columns:
        pasos_df = combined_df.groupby('Tamaño', as_index=False)['Pasos'].max()
        final_df = final_df.merge(pasos_df, on='Tamaño')
else:
    final_df = df
final_df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])


# Cargar los datos desde el archivo CSV exportado
results = pd.read_csv(csv_path)

sizes = results['Tamaño']
times = results['Tiempo (s)']
steps = results['Pasos']

# Manejar la columna de coste si existe
if 'Coste' in results.columns:
    costs = results['Coste']
else:
    import numpy as np
    costs = np.full(len(sizes), np.nan)

fig, axs = plt.subplots(1, 3, figsize=(14, 5))

# Gráfico 1: Tiempo vs Tamaño (sin líneas entre puntos)
axs[0].scatter(sizes, times, marker='o', color='tab:blue')


axs[0].set_xlabel('Tamaño del problema')
axs[0].set_ylabel('Tiempo (s)')
axs[0].set_title('Tiempo vs Tamaño')
axs[0].grid(True)

# Gráfico 2: Pasos vs Tamaño (sin líneas entre puntos)
axs[1].scatter(sizes, steps, marker='s', color='tab:orange')

axs[1].set_xlabel('Tamaño del problema')
axs[1].set_ylabel('Número de pasos')
axs[1].set_title('Pasos vs Tamaño')
axs[1].grid(True)

axs[2].scatter(sizes, costs, marker='^', color='tab:green')
axs[2].set_xlabel('Tamaño del problema')
axs[2].set_ylabel('Coste')
axs[2].set_title('Coste vs Tamaño')
axs[2].grid(True)

plt.tight_layout()
plt.show()



In [ ]:
planner_name = "Fastdownard-seq-opt-bjolp"
sas_output_path = folder_path / "sas_plan"
# Remove --sas-file when using --alias
planner_cmd = [
    "/home/javikugan/Documents/Universidad/Tercero/PlaniExt/Plani/planificadores/downward.sif",
    "--alias", "seq-opt-bjolp", "--plan-file", str(sas_output_path),
    str(folder_path / "dron-domain.pddl")
    # problem_pddl will be appended later
]


max_time = 60
max_size = 0
sizes = []
times = []
steps_list = []
costs = []
outputs = []

size = 2
time_exceeded = False

if not problem_path.exists():
    os.makedirs(problem_path)

pbar = tqdm(desc="Resolviendo tamaños", unit="problema", position=0, leave=True)
while not time_exceeded:
    pbar.set_description(f"Tamaño actual: {size}")
    # Generar el problema con los parámetros actuales
    gen_cmd = f"python3 {folder_path}/generate_problem.py -d 1 -r 1 -l {size} -p {size} -c {size} -g {size} -v 0"
    os.system(gen_cmd)
    pddl_files = sorted(problem_path.glob("*.pddl"), key=os.path.getmtime, reverse=True)
    if not pddl_files:
        break
    problem_pddl = pddl_files[0]
    
    # Construir el comando completo para el planificador
    full_cmd = planner_cmd + [str(problem_pddl)]
    
    pddl_files = sorted(
        problem_path.glob("*.pddl"),
        key=os.path.getmtime,
        reverse=True
    )
    
    if not pddl_files:
        pbar.write("[ERROR] No se encontró el archivo .pddl generado.")
        break
    problem_pddl = pddl_files[0]

    pbar.set_postfix({"Planificador": planner_name, "Tamaño": size})

    start = time.time()
    try:
        result = subprocess.run(full_cmd, timeout=max_time, capture_output=True, text=True)
        elapsed = time.time() - start
        if result.returncode == 0:
    # Encontrar el último archivo sas_plan generado
            sas_files = sorted(
                folder_path.glob("sas_plan*"),
                key=lambda f: int(f.name.split(".")[-1]) if f.name != "sas_plan" else 0,
                reverse=True
            )
            
            if not sas_files:
                pbar.write("[ERROR] No se encontró archivo de plan")
                continue
            
            latest_sas = sas_files[0]
            
            # Leer y procesar el último plan
            plan_output = latest_sas.read_text().strip()
            
            # Contar pasos válidos
            num_steps = sum(
                1 for line in plan_output.split("\n")
                if line.strip() and not line.startswith(";") and "(" in line
            )
            
            # Extraer coste del plan
            plan_cost = None
            for line in plan_output.splitlines():
                if line.startswith(";") and "cost" in line:
                    try:
                        plan_cost = float(line.split("=")[1].split()[0].strip())
                    except (IndexError, ValueError):
                        pass
                    break
                    
            # Eliminar el archivo después de procesarlo
            sizes.append(size)
            times.append(elapsed)
            steps_list.append(num_steps - 1)
            costs.append(plan_cost)
            outputs.append(plan_output)  # guardar solo primeros 500 caracteres
            max_size = size
            
        else:
            pbar.write(f"[ERROR] Planificador {planner_name} falló para tamaño {size}.")
            print(f"Salida del planificador:\n{result.stdout}")
            time_exceeded = True
            break
    except Exception as e:
        pbar.write(f"[ERROR] Excepción al ejecutar el planificador: {e}")
        time_exceeded = True
        break
    pbar.update(1)
    size += 1
    os.remove(problem_pddl)
pbar.close()

print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list,
    "Coste": costs
})

df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Coste": costs,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])



In [ ]:
print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list
})
if csv_path.exists():
    old_df = pd.read_csv(csv_path)
    combined_df = pd.concat([old_df, df], ignore_index=True)
    final_df = combined_df.groupby('Tamaño', as_index=False)['Tiempo (s)'].mean()
    if 'Pasos' in combined_df.columns:
        pasos_df = combined_df.groupby('Tamaño', as_index=False)['Pasos'].max()
        final_df = final_df.merge(pasos_df, on='Tamaño')
else:
    final_df = df
final_df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])


# Cargar los datos desde el archivo CSV exportado
results = pd.read_csv(csv_path)

sizes = results['Tamaño']
times = results['Tiempo (s)']
steps = results['Pasos']

# Manejar la columna de coste si existe
if 'Coste' in results.columns:
    costs = results['Coste']
else:
    import numpy as np
    costs = np.full(len(sizes), np.nan)

fig, axs = plt.subplots(1, 3, figsize=(14, 5))

# Gráfico 1: Tiempo vs Tamaño (sin líneas entre puntos)
axs[0].scatter(sizes, times, marker='o', color='tab:blue')


axs[0].set_xlabel('Tamaño del problema')
axs[0].set_ylabel('Tiempo (s)')
axs[0].set_title('Tiempo vs Tamaño')
axs[0].grid(True)

# Gráfico 2: Pasos vs Tamaño (sin líneas entre puntos)
axs[1].scatter(sizes, steps, marker='s', color='tab:orange')

axs[1].set_xlabel('Tamaño del problema')
axs[1].set_ylabel('Número de pasos')
axs[1].set_title('Pasos vs Tamaño')
axs[1].grid(True)

axs[2].scatter(sizes, costs, marker='^', color='tab:green')
axs[2].set_xlabel('Tamaño del problema')
axs[2].set_ylabel('Coste')
axs[2].set_title('Coste vs Tamaño')
axs[2].grid(True)

plt.tight_layout()
plt.show()



In [ ]:
planner_name = "Fastdownard-seq-opt-fdss-2"
sas_output_path = folder_path / "sas_plan"
# Remove --sas-file when using --alias
planner_cmd = [
    "/home/javikugan/Documents/Universidad/Tercero/PlaniExt/Plani/planificadores/downward.sif",
    "--alias", "seq-opt-fdss-2", "--plan-file", str(sas_output_path),
    str(folder_path / "dron-domain.pddl")
    # problem_pddl will be appended later
]


max_time = 60
max_size = 0
sizes = []
times = []
steps_list = []
costs = []
outputs = []

size = 2
time_exceeded = False

if not problem_path.exists():
    os.makedirs(problem_path)

pbar = tqdm(desc="Resolviendo tamaños", unit="problema", position=0, leave=True)
while not time_exceeded:
    pbar.set_description(f"Tamaño actual: {size}")
    # Generar el problema con los parámetros actuales
    gen_cmd = f"python3 {folder_path}/generate_problem.py -d 1 -r 1 -l {size} -p {size} -c {size} -g {size} -v 0"
    os.system(gen_cmd)
    pddl_files = sorted(problem_path.glob("*.pddl"), key=os.path.getmtime, reverse=True)
    if not pddl_files:
        break
    problem_pddl = pddl_files[0]
    
    # Construir el comando completo para el planificador
    full_cmd = planner_cmd + [str(problem_pddl)]
    
    pddl_files = sorted(
        problem_path.glob("*.pddl"),
        key=os.path.getmtime,
        reverse=True
    )
    
    if not pddl_files:
        pbar.write("[ERROR] No se encontró el archivo .pddl generado.")
        break
    problem_pddl = pddl_files[0]

    pbar.set_postfix({"Planificador": planner_name, "Tamaño": size})

    start = time.time()
    try:
        result = subprocess.run(full_cmd, timeout=max_time, capture_output=True, text=True)
        elapsed = time.time() - start
        if result.returncode == 0:
    # Encontrar el último archivo sas_plan generado
            sas_files = sorted(
                folder_path.glob("sas_plan*"),
                key=lambda f: int(f.name.split(".")[-1]) if f.name != "sas_plan" else 0,
                reverse=True
            )
            
            if not sas_files:
                pbar.write("[ERROR] No se encontró archivo de plan")
                continue
            
            latest_sas = sas_files[0]
            
            # Leer y procesar el último plan
            plan_output = latest_sas.read_text().strip()
            
            # Contar pasos válidos
            num_steps = sum(
                1 for line in plan_output.split("\n")
                if line.strip() and not line.startswith(";") and "(" in line
            )
            
            # Extraer coste del plan
            plan_cost = None
            for line in plan_output.splitlines():
                if line.startswith(";") and "cost" in line:
                    try:
                        plan_cost = float(line.split("=")[1].split()[0].strip())
                    except (IndexError, ValueError):
                        pass
                    break
                    
            # Eliminar el archivo después de procesarlo
            sizes.append(size)
            times.append(elapsed)
            steps_list.append(num_steps - 1)
            costs.append(plan_cost)
            outputs.append(plan_output)  # guardar solo primeros 500 caracteres
            max_size = size
            
        else:
            pbar.write(f"[ERROR] Planificador {planner_name} falló para tamaño {size}.")
            print(f"Salida del planificador:\n{result.stdout}")
            time_exceeded = True
            break
    except Exception as e:
        pbar.write(f"[ERROR] Excepción al ejecutar el planificador: {e}")
        time_exceeded = True
        break
    pbar.update(1)
    size += 1
    os.remove(problem_pddl)
pbar.close()

print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list,
    "Coste": costs
})

df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Coste": costs,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])



In [ ]:
print("\n=== Resumen de resultados ===\n")
print(f"Tamaño máximo resuelto por {planner_name}: {max_size}")

csv_path = folder_path / f"Resultados/resultados_{planner_name.lower()}.csv"
df = pd.DataFrame({
    "Tamaño": sizes,
    "Tiempo (s)": times,
    "Pasos": steps_list
})
if csv_path.exists():
    old_df = pd.read_csv(csv_path)
    combined_df = pd.concat([old_df, df], ignore_index=True)
    final_df = combined_df.groupby('Tamaño', as_index=False)['Tiempo (s)'].mean()
    if 'Pasos' in combined_df.columns:
        pasos_df = combined_df.groupby('Tamaño', as_index=False)['Pasos'].max()
        final_df = final_df.merge(pasos_df, on='Tamaño')
else:
    final_df = df
final_df.to_csv(csv_path, index=False)
print(f"Resultados de {planner_name} exportados a {csv_path}")

if sizes:
    df_display = pd.DataFrame({
        "Tamaño": sizes,
        "Tiempo (s)": times,
        "Pasos plan": steps_list,
        "Planificador": planner_name
    })
    display(df_display)

if outputs:
    max_idx = sizes.index(max_size)
    print(f"\nSalida fragmento para {planner_name} tamaño {max_size}:\n")
    print(outputs[max_idx])


# Cargar los datos desde el archivo CSV exportado
results = pd.read_csv(csv_path)

sizes = results['Tamaño']
times = results['Tiempo (s)']
steps = results['Pasos']

# Manejar la columna de coste si existe
if 'Coste' in results.columns:
    costs = results['Coste']
else:
    import numpy as np
    costs = np.full(len(sizes), np.nan)

fig, axs = plt.subplots(1, 3, figsize=(14, 5))

# Gráfico 1: Tiempo vs Tamaño (sin líneas entre puntos)
axs[0].scatter(sizes, times, marker='o', color='tab:blue')


axs[0].set_xlabel('Tamaño del problema')
axs[0].set_ylabel('Tiempo (s)')
axs[0].set_title('Tiempo vs Tamaño')
axs[0].grid(True)

# Gráfico 2: Pasos vs Tamaño (sin líneas entre puntos)
axs[1].scatter(sizes, steps, marker='s', color='tab:orange')

axs[1].set_xlabel('Tamaño del problema')
axs[1].set_ylabel('Número de pasos')
axs[1].set_title('Pasos vs Tamaño')
axs[1].grid(True)

axs[2].scatter(sizes, costs, marker='^', color='tab:green')
axs[2].set_xlabel('Tamaño del problema')
axs[2].set_ylabel('Coste')
axs[2].set_title('Coste vs Tamaño')
axs[2].grid(True)

plt.tight_layout()
plt.show()



# Resultados Finales

In [ ]:
#    
    

# Gráfico 1: Tiempo vs Tamaño
plt.subplot(1, 2, 1)
for df, label, color, marker in [
    (ff_df, 'FF', 'blue', 'o'),
    (lpg_df, 'LPG-TD', 'orange', 's'),
    (sgplan_df, 'SGPLAN40', 'green', '^')
]:
    plt.scatter(df['Tamaño'], df['Tiempo (s)'], label=label, 
                marker=marker, color=color)

plt.xlabel('Tamaño del problema')
plt.ylabel('Tiempo (s)')
plt.title('Tiempo vs Tamaño con Regresión Lineal')
plt.legend()
plt.grid(True)

# Gráfico 2: Pasos vs Tamaño
plt.subplot(1, 2, 2)
for df, label, color, marker in [
    (ff_df, 'FF', 'blue', 'o'),
    (lpg_df, 'LPG-TD', 'orange', 's'),
    (sgplan_df, 'SGPLAN40', 'green', '^')
]:
    plt.scatter(df['Tamaño'], df['Pasos'], label=label, 
                marker=marker, color=color)

plt.xlabel('Tamaño del problema')
plt.ylabel('Número de pasos')
plt.title('Pasos vs Tamaño con Regresión Lineal')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Mostrar estadísticas después de los gráficos
print("\n" + "="*50 + "\nAnálisis Estadístico Comparativo\n" + "="*50)
for df, name in [(ff_df, 'FF'), (lpg_df, 'LPG-TD'), (sgplan_df, 'SGPLAN40')]:
    print_statistics(df, name)
